In [ ]:
"""
※ 참고 사이트
- 얼굴 인식하여 잠금 해제
    https://blog.naver.com/chandong83/221695462391
"""

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isdir, isfile, join

In [12]:
face_path = './98_LearningData/04_StudyData/MultiFace/'
train_path = './98_LearningData/51_Learning_Data/01_OpenCv/MultiFace/'
face_cascades = cv2.CascadeClassifier('./98_LearningData/00_haarcascades/haarcascade_frontalface_default.xml')
eye_cascades = cv2.CascadeClassifier('./98_LearningData/00_haarcascades/haarcascade_lefteye_2splits.xml')

In [13]:
# 사용자 얼굴 학습
def train(name):
    # 모델 생성
    model = cv2.face.LBPHFaceRecognizer_create()
    
    #학습한 파일이 있는지 체크
    if isfile(join(train_path, name + '.yml')):
        model.read(join(train_path, name + '.yml'))
        print("Train data exists!!")
    #학습한 파일이 없을 경우
    else:    
        data_path = face_path + name + '/'
        #파일만 리스트로 만듬
        face_pics = [f for f in listdir(data_path) if isfile(join(data_path,f))]

        Training_Data, Labels = [], []

        for i, files in enumerate(face_pics):
            image_path = data_path + face_pics[i]
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # 이미지가 아니면 패스
            if images is None:
                continue    
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
            
        if len(Labels) == 0:
            print("There is no data to train.")
            return None
        Labels = np.asarray(Labels, dtype=np.int32)
        
        # 학습
        model.train(np.asarray(Training_Data), np.asarray(Labels))
        #학습 저장
        model.save(join(train_path, name + '.yml'))
        print(name + " : Model Training Complete!!!!!")

    #학습 모델 리턴
    return model

In [8]:
# 여러 사용자 학습
def trains():
    # 폴더만 색출
    model_dirs = [f for f in listdir(face_path) if isdir(join(face_path,f))]
    
    #학습 모델 저장할 딕셔너리
    models = {}
    # 각 폴더에 있는 얼굴들 학습
    for model in model_dirs:
        print('model :' + model)
        
        # 학습 시작
        result = train(model)
        
        # 학습이 안되었다면 패스!
        if result is None:
            continue
            
        # 학습되었으면 저장
        models[model] = result

    # 학습된 모델 딕셔너리 리턴
    return models  

In [16]:
#얼굴 검출
def face_detector(img, size = 0.5):
    #이미지를 흑백으로 변경
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #얼굴 검출
    faces = face_cascades.detectMultiScale(grayscale_img, scaleFactor=1.1, minNeighbors=5, minSize=(100,100), flags=cv2.CASCADE_SCALE_IMAGE)
    
    if faces is():
        return img,[]
    
    for(x,y,w,h) in faces:
        #눈을 추려내기 위해서 이미지 크롭
        img_trim = img[y:y+h, x:x+w]
        
        #눈 추려내기
        eye_list = eye_cascades.detectMultiScale(img_trim,scaleFactor = 1.15, minNeighbors=1,minSize=(10,10))
        
        #눈이 있으면서 2개 이상)
        if len(eye_list) >= 2:
            #얼굴에 네모 표기하기
            cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)

            #눈에 네모 표기 하기(눈에표기하면 인식 안됨 네모도 얼굴의 일부로 인식)
            #for eye in eye_list:
                #eyeX,eyeY,eyeW,eyeH = eye
                #cv2.rectangle(img, (eyeX + x, eyeY + y), (eyeX + eyeW + x, eyeY + eyeH + y), (0, 255, 0), thickness=8)

            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200,200))
        else:
            return img,[]
    return img,roi   #검출된 좌표에 사각 박스 그리고(img), 검출된 부위를 잘라(roi) 전달


In [17]:
# 인식 시작
def run(models):    
    #카메라 열기 
    cap = cv2.VideoCapture(0)
    
    while True:
        #카메라로 부터 사진 한장 읽기 
        ret, frame = cap.read()
        # 얼굴 검출 시도 
        image, face = face_detector(frame)
        try:            
            min_score = 999       #가장 낮은 점수로 예측된 사람의 점수
            min_score_name = ""   #가장 높은 점수로 예측된 사람의 이름
            
            #검출된 사진을 흑백으로 변환 
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            #위에서 학습한 모델로 예측시도 - 제일 낮은 사람의 결과를 가져오기 위한 부분
            for key, model in models.items():
                result = model.predict(face)                
                if min_score > result[1]:
                    min_score = result[1]
                    min_score_name = key
                    
            #min_score 신뢰도이고 0에 가까울수록 자신과 같다는 뜻이다.         
            if min_score < 500:
                #% 처리하기 위함
                confidence = int(100*(1-(min_score)/300))
                # 유사도 화면에 표시 
                display_string = str(confidence)+'% Confidence it is ' + min_score_name
            
            #가장 높은 유사도의 값을 화면에 표시
            cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
            
            #75 보다 크면 Success
            if confidence > 75:
                cv2.putText(image, "Unlocked : " + min_score_name, (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face Cropper', image)
            #75 이하면 Fail
            else:
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Face Cropper', image)
        except:
            #얼굴 검출 안됨 
            cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('Face Cropper', image)
            pass
        if cv2.waitKey(1)==13:
            break
    cap.release()
    cv2.destroyAllWindows()

In [20]:
#__main__을 넣는 이유는 이 프로그램이 최초 실행인지 import한건지 체크하기 위함
if __name__ == "__main__":
    # 학습 시작
    models = trains()
    # 고!
    run(models)

model :19840216
Train data exists!!
model2 :19840216
model :19840902
Train data exists!!
model2 :19840902
model :20120723
Train data exists!!
model2 :20120723


In [8]:
print(__name__)

__main__
